In [3]:
import requests, hashlib, os
from datetime import datetime

# Dossier où tu stockes tes sources
dossier = "dataset"
os.makedirs(dossier, exist_ok=True)

# Chemins utiles
fichier_temp = os.path.join(dossier, "temp_dvf.csv.gz")
fichier_hash = os.path.join(dossier, "dernier_hash.txt")

# URL stable du fichier DVF géolocalisé
url = "https://www.data.gouv.fr/fr/datasets/r/d7933994-2c66-4131-a4da-cf7cd18040a4"

# 1. Télécharger temporairement
print("📥 Téléchargement en cours...")
r = requests.get(url, stream=True)
with open(fichier_temp, 'wb') as f:
    for chunk in r.iter_content(8192):
        f.write(chunk)

# 2. Calcul de l'empreinte SHA1
def sha1(path):
    h = hashlib.sha1()
    with open(path, 'rb') as f:
        while chunk := f.read(8192):
            h.update(chunk)
    return h.hexdigest()

nouveau_hash = sha1(fichier_temp)
ancien_hash = open(fichier_hash).read().strip() if os.path.exists(fichier_hash) else None

# 3. Comparaison & archivage
if nouveau_hash != ancien_hash:
    nom_final = os.path.join(dossier, f"dvf_{datetime.today().strftime('%Y%m%d')}.csv.gz")
    os.rename(fichier_temp, nom_final)
    with open(fichier_hash, 'w') as f:
        f.write(nouveau_hash)
    print(f"✅ Nouveau fichier détecté et sauvegardé : {nom_final}")
else:
    os.remove(fichier_temp)
    print("✔️ Aucun changement détecté. Le fichier est inchangé.")
    
import pandas as pd

# Chargement direct dans un DataFrame
print("🔍 Chargement du fichier dans un DataFrame...")
df = pd.read_csv(nom_final, compression='gzip', low_memory=False)

# Aperçu rapide
print(f"{len(df):,} lignes chargées.")
print("Colonnes disponibles :", list(df.columns))

# Suppression des colonnes adresse_suffixe, ancien_code_commune, ancien_nom_commune, ancien_id_parcelle, numero_volume, nature_culture_speciale, code_nature_culture
df = df.drop(columns=['adresse_suffixe', 'ancien_code_commune', 'ancien_nom_commune', 'ancien_id_parcelle', 'numero_volume','nature_culture_speciale', 'code_nature_culture'])

# Suppression des lignes où la valeur fonciere est nulle:
df = df.dropna(subset=['valeur_fonciere'])

# Suppression des lignes où la longitude et latitude sont nulles:
df=df.dropna(subset=['longitude'])

# Suppression des lignes missing de la colonne 'type_local' afin de garder uniquement maison, appartement, dependances et local:
df = df.dropna(subset=['type_local'])

# suppression des colonnes lot1_numero, lot1_surface_carrez, lot2_numero, lot2_surface_carrez, lot3_numero, lot3_surface_carrez, lot4_numero, lot4_surface_carrez, lot5_numero, lot5_surface_carrez, nombre_lots:
df = df.drop(columns=['lot1_numero', 'lot1_surface_carrez','lot2_numero', 'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez', 'lot4_numero', 'lot4_surface_carrez', 'lot5_numero', 'lot5_surface_carrez', 'nombre_lots']) 

#suppression des type_local qui ne sont pas maison ou appartement: 
df_filtré = df[df["type_local"].isin(["Maison", "Appartement"])]

#suppression des nature mutations qui ne sont pas vente ou vente en l'etat futur d'achevement: 
df_filtré = df_filtré[df_filtré["nature_mutation"].isin(["Vente", "Vente en l'etat futur d'achèvement"])]

#suppression de la colonne code_nature_culture_speciale:
df_filtré = df_filtré.drop(columns=['code_nature_culture_speciale'])

#suppression des colonnes surface_terrain et nature_culture:
df_filtré = df_filtré.drop(columns=['surface_terrain', 'nature_culture'])

#suppression des lignes doublons:
df_filtré = df_filtré.drop_duplicates()

# Conservons uniquement une ligne unique par id_mutation avec somme du nombre de pieces et somme de la surface batie:

df_agrégé = df_filtré.groupby('id_mutation', as_index=False).agg({
    'date_mutation': 'first',
    'numero_disposition': 'first',
    'nature_mutation': 'first',
    'valeur_fonciere': 'first',
    'adresse_numero': 'first',
    'adresse_nom_voie': 'first',
    'adresse_code_voie': 'first',
    'code_postal': 'first',
    'code_commune': 'first',
    'nom_commune': 'first',
    'code_departement': 'first',
    'id_parcelle': 'first',
    'code_type_local': 'first',
    'surface_reelle_bati': 'sum',
    'nombre_pieces_principales': 'sum',
    'type_local': 'first',
    'longitude': 'first',
    'latitude': 'first',
})

# creation d'une nouvelle colonne prix_m2:
df_agrégé['prix_m2'] = df_agrégé['valeur_fonciere'] / df_agrégé['surface_reelle_bati']

# Supprimer les lignes avec des prix_m2 infinis:

df_agrégé= df_agrégé[~df_agrégé['prix_m2'].isin([float('inf'), float('-inf')])]  # enlève les inf

# Conversion vers int après nettoyage
df_agrégé['prix_m2'] = df_agrégé['prix_m2'].astype(int)

#suppression des lignes pour lesquelles le prix_m2 est egal à 0:
df_agrégé = df_agrégé[df_agrégé['prix_m2'] != 0]

# transformation de la colonne date_mutation en type datetime:
df_agrégé['date_mutation'] = pd.to_datetime(df_agrégé['date_mutation'])

#sauvegarde en fichier csv:
df_agrégé.to_csv(os.path.join(dossier, 'df_agrégé.csv'), index=False)


📥 Téléchargement en cours...
✅ Nouveau fichier détecté et sauvegardé : dataset\dvf_20250702.csv.gz
🔍 Chargement du fichier dans un DataFrame...
20,133,668 lignes chargées.
Colonnes disponibles : ['id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation', 'valeur_fonciere', 'adresse_numero', 'adresse_suffixe', 'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune', 'nom_commune', 'code_departement', 'ancien_code_commune', 'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle', 'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero', 'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez', 'lot4_numero', 'lot4_surface_carrez', 'lot5_numero', 'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local', 'surface_reelle_bati', 'nombre_pieces_principales', 'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale', 'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude']
